In [6]:
import pandas as pd

# read in ratings
ratings = pd.read_csv(r"C:\Users\HomePC\Desktop\envoypathways\ratings.csv")

# read in movie details
movies = pd.read_csv(r"C:\Users\HomePC\Desktop\envoypathways\movies.csv")

# Merge both datasets
df = pd.merge(ratings, movies, on="movieId")
print(df.head())

   userId  movieId  rating  timestamp                        title  \
0       1        1     4.0  964982703             Toy Story (1995)   
1       1        3     4.0  964981247      Grumpier Old Men (1995)   
2       1        6     4.0  964982224                  Heat (1995)   
3       1       47     5.0  964983815  Seven (a.k.a. Se7en) (1995)   
4       1       50     5.0  964982931   Usual Suspects, The (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                               Comedy|Romance  
2                        Action|Crime|Thriller  
3                             Mystery|Thriller  
4                       Crime|Mystery|Thriller  


In [7]:
user_item_matrix = df.pivot_table(index="userId", columns="title", values="rating")

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Fill missing ratings with 0
user_item_filled = user_item_matrix.fillna(0)

# get  similarity
user_similarity = cosine_similarity(user_item_filled)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

In [12]:
def recommend_movies(user_id, n=5):
    # Get similarity scores for the user
    sim_scores = user_similarity_df[user_id].sort_values(ascending=False)
    
    # Exclude the user themselves
    sim_scores = sim_scores.drop(user_id)
    
    # Pick the most similar user
    most_similar_user = sim_scores.idxmax()
    
    # Movies rated by similar user
    movies_watched = user_item_matrix.loc[user_id].dropna().index
    recommendations = user_item_matrix.loc[most_similar_user].dropna().sort_values(ascending=False)
    
    # Recommend movies not seen by the target user
    recommendations = recommendations.drop(movies_watched, errors="ignore")
    
    return recommendations.head(n)

print(recommend_movies(user_id=1, n=5))


title
2001: A Space Odyssey (1968)                    5.0
Aliens (1986)                                   5.0
Austin Powers: The Spy Who Shagged Me (1999)    5.0
Commitments, The (1991)                         5.0
Casino (1995)                                   5.0
Name: 266, dtype: float64


In [13]:
def precision_at_k(recommended, actual, k):
    recommended_k = recommended[:k]
    relevant = set(recommended_k).intersection(set(actual))
    return len(relevant) / k